In [1]:
import torch
import torch.nn as nn
import time 
from time import sleep
import datetime
import os 
import cv2
import numpy as np 
import torch.optim as optim
import matplotlib.pyplot as plt 
from torch.utils.data import DataLoader 
from sklearn.model_selection import train_test_split
import pandas as pd
import TrainHelper as th
import CustomModels as cm
import CustomHubModels as chm
import CustomImageDataset as cid
import torch.nn.functional as F
import LossFunction as lf 
import HubLossFunctions as hlf
from torchvision.io import read_image, ImageReadMode
device = ("cuda" if torch.cuda.is_available() else "cpu")

import sys
sys.path.append(os.path.join('..','segment-utils'))
import VideoAnnotationHelper as vah
from torchinfo import summary

C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please

In [2]:
parent_dir = "C:/Lumena"
train_data_folder = "train_data"
sub_folders = ["EL", "PC2", "PR", "EL_2", "CNMC", "CNMC.2023.11.07", "SS", "Bob", "PF"]
csv_filename = ["cropped_256x256.csv","cropped_256x256_original.csv"]
img_dir = ["images_cropped_256x256", "images_cropped_256x256_original"]
mask_dir = ["masksLined_cropped_256x256","masksLined_cropped_256x256_original"]

batch_size = 128
epochs = 400
cycles = 1 

encoder_lr = 1e-4  
regA_lr = 1e-4
# regB_lr = 1e-4

stop_encoder_requires_grad = False  
stop_regA_requires_grad = False 
# stop_regB_requires_grad = True 

FullRange = False 
Gray = False
ifTrain = True 
ifMSKCha = False 

observe_path="D:/test_dir"

old_model_path = "//cgmqnap.clearguide.local/data/Needles/Lumena/models/hub_regression/model_cyc_1_ep_400_bs_32_lr_0.0001_20231228_T171234.pt"

In [3]:
images_path = []
labels = []

for k in range(len(csv_filename)):
    dfs = []
    for ele in sub_folders:
        df = pd.read_csv(parent_dir + "/" + train_data_folder + "/" + ele +"/" + csv_filename[k])
        dfs.append(df)

    for i in range(len(dfs)): 
        for name in zip(dfs[i]["Images"], dfs[i]["Hub Y"], dfs[i]["Hub X"], dfs[i]["Tip Y"], dfs[i]["Tip X"],dfs[i]["Hub Present"]): 
            if name[3]:
                label_ele = [] 
                images_path.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + img_dir[k] + "/" + name[0])
                label_ele.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + mask_dir[k] + "/" + name[0])
                label_ele.append(name[1:])
                labels.append(label_ele)
      
    
X_train, X_val, y_train, y_val = train_test_split(images_path, labels, test_size = 0.2, random_state = 24)

train_dataset = cid.Custom_ImageRegressor_Dataset2(X_train, y_train, ifgray = Gray, ifTrain = ifTrain, ifFullRange = FullRange, ifAddMskCha = ifMSKCha)
val_dataset = cid.Custom_ImageRegressor_Dataset2(X_val, y_val, ifgray = Gray, ifTrain = ifTrain, ifFullRange = FullRange, ifAddMskCha = ifMSKCha)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True,num_workers=4,pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False,num_workers=4,pin_memory = True)

In [4]:
model_name = "model" 
model_name += "_cyc_" + str(cycles) + "_ep_" + str(epochs) + "_bs_" + str(batch_size)
model_name += "_lr_" + str(regA_lr)
model_folder = model_name +  "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now())
 
vah.createFolder(observe_path + "/"  + model_folder)

Created folder 'D:/test_dir/model_cyc_1_ep_400_bs_128_lr_0.0001_20240115_T105104'.


In [5]:
model = cm.TipHubRegression().to(device)
summary(model)

Layer (type:depth-idx)                                                 Param #
TipHubRegression                                                       --
├─Sequential: 1-1                                                      --
│    └─ConvBlockReLU: 2-1                                              --
│    │    └─Sequential: 3-1                                            2,400
│    └─ConvBlockReLU: 2-2                                              --
│    │    └─Sequential: 3-2                                            12,592
├─Sequential: 1-2                                                      --
│    └─ConvBlockReLU: 2-3                                              --
│    │    └─Sequential: 3-3                                            12,896
│    └─ConvBlockReLU: 2-4                                              --
│    │    └─Sequential: 3-4                                            25,696
├─Sequential: 1-3                                                      --
│    └─InceptionBl

In [6]:
# old_model = torch.jit.load(old_model_path)
# summary(old_model)

In [7]:
# model.load_state_dict(old_model.state_dict(), strict=False) 

In [8]:
 i = 0
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print("NO.", i, name)
        i += 1

NO. 0 conv1.0.conv_block.0.conv_block.0.weight
NO. 1 conv1.0.conv_block.0.conv_block.0.bias
NO. 2 conv1.0.conv_block.0.conv_block.1.weight
NO. 3 conv1.0.conv_block.0.conv_block.1.bias
NO. 4 conv1.1.conv_block.0.conv_block.0.weight
NO. 5 conv1.1.conv_block.0.conv_block.0.bias
NO. 6 conv1.1.conv_block.0.conv_block.1.weight
NO. 7 conv1.1.conv_block.0.conv_block.1.bias
NO. 8 conv2.0.conv_block.0.conv_block.0.weight
NO. 9 conv2.0.conv_block.0.conv_block.0.bias
NO. 10 conv2.0.conv_block.0.conv_block.1.weight
NO. 11 conv2.0.conv_block.0.conv_block.1.bias
NO. 12 conv2.1.conv_block.0.conv_block.0.weight
NO. 13 conv2.1.conv_block.0.conv_block.0.bias
NO. 14 conv2.1.conv_block.0.conv_block.1.weight
NO. 15 conv2.1.conv_block.0.conv_block.1.bias
NO. 16 conv3.0.layer.0.group1.conv_block.0.weight
NO. 17 conv3.0.layer.0.group1.conv_block.0.bias
NO. 18 conv3.0.layer.0.group1.conv_block.1.weight
NO. 19 conv3.0.layer.0.group1.conv_block.1.bias
NO. 20 conv3.0.layer.0.group2.0.conv_block.0.conv_block.0.weig

In [9]:
 # lossFun1 = nn.MSELoss()
#lossFun1 = lf.MSE 
LossFun1 = hlf.LineDistanceLoss() 
LossFun2 = lf.MSE

In [10]:
 i = 0 
encoder_list = []
regressorA_list = []
regressorB_list = [] 

for param in model.parameters(): 
    if i < 180: 
        if stop_encoder_requires_grad:
            param.requires_grad = False 
        encoder_list.append(param)
    else: 
        if stop_regA_requires_grad: 
            param.requires_grad = False 
        regressorA_list.append(param)
        
    i += 1

In [11]:
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print(name)

conv1.0.conv_block.0.conv_block.0.weight
conv1.0.conv_block.0.conv_block.0.bias
conv1.0.conv_block.0.conv_block.1.weight
conv1.0.conv_block.0.conv_block.1.bias
conv1.1.conv_block.0.conv_block.0.weight
conv1.1.conv_block.0.conv_block.0.bias
conv1.1.conv_block.0.conv_block.1.weight
conv1.1.conv_block.0.conv_block.1.bias
conv2.0.conv_block.0.conv_block.0.weight
conv2.0.conv_block.0.conv_block.0.bias
conv2.0.conv_block.0.conv_block.1.weight
conv2.0.conv_block.0.conv_block.1.bias
conv2.1.conv_block.0.conv_block.0.weight
conv2.1.conv_block.0.conv_block.0.bias
conv2.1.conv_block.0.conv_block.1.weight
conv2.1.conv_block.0.conv_block.1.bias
conv3.0.layer.0.group1.conv_block.0.weight
conv3.0.layer.0.group1.conv_block.0.bias
conv3.0.layer.0.group1.conv_block.1.weight
conv3.0.layer.0.group1.conv_block.1.bias
conv3.0.layer.0.group2.0.conv_block.0.conv_block.0.weight
conv3.0.layer.0.group2.0.conv_block.0.conv_block.0.bias
conv3.0.layer.0.group2.0.conv_block.0.conv_block.1.weight
conv3.0.layer.0.grou

In [18]:
encoder_lr = 1e-5
regA_lr = 1e-4

In [19]:
for j in range(cycles):

    print("Training Cycle: ", j+1)

    optimizer = torch.optim.Adam([{"params":encoder_list, "lr":encoder_lr},
                                  #{"params":regressorA_list, "lr":regA_lr},
                                  #{"params":regressorB_list, "lr":regB_lr}
                                 ])#, weight_decay = 1e-5)
    
    
#     early_stopping = th.EarlyStopping(patience = 10, min_delta = 0.001)
    
    
    for epoch in range(epochs): 
        
        train_loss = 0.0
        val_loss = 0.0        
        start_time = time.time() 
        
        model.train()
        for data in train_loader:
            images, true_yx, true_p= data 
            timex = time.process_time()
            images = images.to(device)
            true_yx = true_yx.to(device)

            optimizer.zero_grad()
            
            pred_yx, pred_p = model(images)
            # pred_yx = model(images)
            
            loss = LossFun1.forward(pred_yx.float(),true_yx.float()) + LossFun2(pred_yx.float(),true_yx.float())
            loss.backward()
      
            optimizer.step()
            
            train_loss += loss

            
        model.eval()    
        for data in val_loader: 
            
            images, true_yx, true_p= data
            images = images.to(device)
            true_yx = true_yx.to(device)

            with torch.no_grad(): 
                pred_yx, pred_p = model(images)
                # pred_yx = model(images)
                loss = LossFun1.forward(pred_yx.float(),true_yx.float()) + LossFun2(pred_yx.float(),true_yx.float())
                # loss.backward()
     
            val_loss += loss

        
        train_loss = train_loss/len(train_loader)    
        val_loss = val_loss/len(val_loader)
        
        
        print('Epoch: {}  \tTraining Loss: {:.6f} \t Vali Loss: {:.6f} \t Exe Time: {:.6f}sec'.format(
        epoch, 
        train_loss, 
        val_loss,
        time.time() - start_time))

#         stop = early_stopping.early_stop(val_loss, model)
#         if stop: 
#             model = early_stopping.get_model() 
#             print("Early Stopping because of regressor val loss")
#             break 
        
        if epoch % 10 == 0: 
 
            model_script = torch.jit.script(model)
            temp_name = model_name + "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now()) + ".pt"
            model_script.save(observe_path + "/"  + model_folder + "/" + temp_name)
            print("Temprary model saved: ", temp_name)             

Training Cycle:  1
Epoch: 0  	Training Loss: 0.038832 	 Vali Loss: 0.067894 	 Exe Time: 18.033066sec
Temprary model saved:  model_cyc_1_ep_400_bs_128_lr_0.0001_20240116_T132308.pt
Epoch: 1  	Training Loss: 0.046202 	 Vali Loss: 0.070302 	 Exe Time: 16.513996sec
Epoch: 2  	Training Loss: 0.045626 	 Vali Loss: 0.068516 	 Exe Time: 16.739893sec
Epoch: 3  	Training Loss: 0.056010 	 Vali Loss: 0.073477 	 Exe Time: 16.684041sec
Epoch: 4  	Training Loss: 0.039011 	 Vali Loss: 0.065758 	 Exe Time: 16.454369sec
Epoch: 5  	Training Loss: 0.036034 	 Vali Loss: 0.065554 	 Exe Time: 16.520090sec
Epoch: 6  	Training Loss: 0.042905 	 Vali Loss: 0.075649 	 Exe Time: 16.339598sec
Epoch: 7  	Training Loss: 0.048460 	 Vali Loss: 0.059764 	 Exe Time: 16.285261sec
Epoch: 8  	Training Loss: 0.041594 	 Vali Loss: 0.069704 	 Exe Time: 16.235466sec
Epoch: 9  	Training Loss: 0.058400 	 Vali Loss: 0.069033 	 Exe Time: 16.289102sec
Epoch: 10  	Training Loss: 0.041542 	 Vali Loss: 0.066155 	 Exe Time: 16.393565sec

KeyboardInterrupt: 